In [1]:
import pandas as pd
import numpy as np
from math import log10

In [2]:
# Oсреднение по вертикали vertical
def vertical(file, data_z):
    Y = file[file['X'] == 0]['Y']
    X = file['X'].unique()
    result = []
    stack = []
    
    for i in range(len(X)):
        for j in range(len(Y)):
            element = data_z[j + i * len(Y)]
            if len(stack) > 1 and (np.mean(stack) * 1.2 <= element or np.mean(stack) * 0.8 >= element):
                average_for_stack = np.mean(stack)
                stack = [average_for_stack] * len(stack)
                result.extend(stack)
                stack.clear()
                stack.append(element)
            else:
                stack.append(element)
        average_for_stack = np.mean(stack)
        stack = [average_for_stack] * len(stack)        
        result.extend(stack)
        stack.clear()
        
    return result

In [3]:
# Осреднение по горизонтали horizontal
def horizontal(file, data_z):
    Y = file[file['X'] == 0]['Y']
    X = file['X'].unique()
    stack = []
    result = []

    # осреднение
    for y in Y:
        row = data_z[file['Y'] == y]
        for element in row:
            if len(stack) > 1 and (np.mean(stack) * 1.2 <= element or np.mean(stack) * 0.8 >= element):
                average_for_stack = np.mean(stack)
                stack = [average_for_stack] * len(stack)
                result.extend(stack)
                stack.clear()
                stack.append(element)
            else:
                stack.append(element)
        average_for_stack = np.mean(stack)
        stack = [average_for_stack] * len(stack)        
        result.extend(stack)
        stack.clear()
    
    # сборка в правильный столбец
    finish = []
    for i in range(len(X)):
        for j in range(len(Y)):
            finish.append(result[j * len(X) + i])
    
    return finish

In [14]:
def bin_averaging(data):
    min_data = min(data)
    max_data = max(data)
    # Поиск значений, к которым осредняем данные
    bins = np.logspace(np.log10(min(data)) - 0.0001, np.log10(max(data)) + 0.0001, num=6)
    key_nums = []
    for i in range(len(bins) - 1):
        number = (bins[i] * bins[i + 1]) ** (1/2)
        key_nums.append(number)
    # Осреднение
    #bins = [12.76, 45.02, 128.61, 367.40, 1049.53, 22998] #2750
    #key_nums = [26.64, 76.10, 217.37, 620.96, 1773.88]    #2750
    bins = [12.76, 45.02, 120, 450.40, 1049.53, 22998]    #3000
    key_nums = [28, 100, 280, 700, 1500]                  #3000
    #bins = [26.4094068, 69.15747178, 181.10046694, 474.24202018, 1241.88246172, 3252.07801734]                    #2500
    #key_nums = [42.736492664845166, 111.91269111019771, 293.062196978768, 767.4326338328589, 2009.6513762036247] #2500
    result = []
    for value in data:
        for i in range(len(bins) - 1):
            if bins[i] <= value < bins[i + 1]:
                result.append(key_nums[i])
    return result

In [15]:
# Здесь задаем файл и столбец с данными + выбор функций

file = pd.read_excel('D:/Learning/pythonich/Работа_с_библиотеками/for_diploma/pr_3000.xlsx', sheet_name='main')
data_z = file['OC']
# file_mq = file[file['Y'] >= -95]
# data_mq = 
# new_data_column = vertical(file, data_z)
# new_data_column = horizontal(file, data_z)
# new_data_column = vertical(file, horizontal(file, data_z))
#new_data_column = horizontal(file, pd.Series(vertical(file, data_z)))
new_data_column = bin_averaging(pd.Series(horizontal(file, pd.Series(vertical(file, data_z)))))
file['result'] = new_data_column

In [16]:
# Сохраняем в Excel
file_path = 'D:/Learning/pythonich/Работа_с_библиотеками/for_diploma/pr_3000.xlsx'
sheet_name = 'main'
with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='replace') as writer:
    file.to_excel(writer, sheet_name=sheet_name, index=False)